### Data preparations

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install neptune lightning pytorchvideo

In [ ]:
import neptune

run = neptune.init_run(
    project="afonchikovd585/Cells-classification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkMmY4NmNkZC05OTJlLTQwZGQtOTAwZC1kYzU1MTUwMzRjMzYifQ==",
)  # your credentials

In [ ]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

import pandas as pd
import numpy as np
import os
import shutil
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, seed_everything, Trainer
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [ ]:
video_transforms = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x: x / 255.0),
        Normalize((0.9133, 0.2737, 0.2737),(0.1576, 0.2508, 0.2508)),
        # RandomShortSideScale(min_size=248, max_size=256),
        # CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [ ]:
dataset_path = "/kaggle/input/cells-dataset-1-frame-wo-alpha/dataset_1_frame"

In [ ]:
os.chdir(dataset_path)
os.chdir("..")

In [ ]:
ns = {'train': 0.6, 'val': 0.2,  'test': 0.2}
# Function for setting the seed
seed = 42 # random seeds are 42, 0, 17, 9, 3

In [ ]:
seed_everything(seed)
dataframe = pd.read_csv("DataFrame1.csv", index_col=0)
# dataframe = dataframe.sample(frac=1)
# ratio = 0.8
# train_size = int(dataframe.shape[0] * ratio)
# train_data = dataframe[0:train_size]
# test_data = dataframe[train_size:]
# train_data, test_data
train_data, val_data, test_data = torch.utils.data.random_split(dataframe, [ns['train'],ns['val'],ns['test']])

In [ ]:
class CustomDataset(LabeledVideoDataset):
    def __init__(self, dataset_path, dataframe, target_name, transforms, clip_sampler_type='random', clip_duration=1):
      df = dataframe.reset_index()
      paths = []
      for i, file_name in enumerate(df['files']):
          temp_dict = df.iloc[i].to_dict()
          temp_dict['label'] = df[target_name][i]
          temp_dict.pop('files')
          temp_dict.pop('index')
          paths.append((f"{dataset_path}/{file_name}", temp_dict))
      super().__init__(labeled_video_paths=paths,
                       clip_sampler=make_clip_sampler(clip_sampler_type, clip_duration),
                       transform=transforms, decode_audio=False)

In [ ]:
# Used for testing theory, ignore that
train_dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                              target_name='high_erythrocytes',
                              transforms=video_transforms)
train_dataset.num_videos

In [ ]:
checkpoint_callback = ModelCheckpoint(monitor="val/metric",
                                      mode="max",
                                      dirpath="checkpoints", 
                                      filename="file", 
                                      save_last=True)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

In [ ]:
class CurriculumTrainer():
    def set_difficulty(self, dataframe, target_cells, target_name, alpha, beta, gamma):
        # Calculating distances between mean and target value
        df = dataframe.copy(deep=True)
        df['distances'] = (df['erythrocytes'] - df['erythrocytes'].mean()).abs()

        # Normalizing blur and distances
        df['blur'] = (df['blur'] - df['blur'].min()) / (df['blur'].max() - df['blur'].min())
        df['distances'] = (df['distances'] - df['distances'].min()) / (df['distances'].max() - df['distances'].min())

        # Calculating and normalizing difficulty
        df['difficulty'] = alpha * df['blur'] + beta * df['noise'] + gamma * df['distances']
        df['difficulty'] = (df['difficulty'] - df['difficulty'].min()) / (df['difficulty'].max() - df['difficulty'].min())
        return df

    def evaluate_competence(self, max_epochs, current_epoch, c0, p):
        return min(1, (current_epoch*((1-c0**p)/max_epochs)+c0**p)**(1/p))

    def fit(self, model, dataframe, target_cells, target_name, max_epochs, c0, p, alpha, beta, gamma):
        dataframe = self.set_difficulty(dataframe, target_cells, target_name, alpha, beta, gamma)
        self.competence = c0
        seed_everything(seed)
        for epoch in range(1, max_epochs+1):
            selected_data = dataframe[dataframe.difficulty <= self.competence]
            dataset = CustomDataset(dataset_path=dataset_path, dataframe=selected_data,
                                    target_name=target_name,
                                    transforms=video_transforms)
            loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
            print(f"-----------------------\nEpoch {epoch}, competence = {self.competence}, dataset size = {dataset.num_videos}")
            run["model/competence"].append(self.competence)
            run["train/dataset_size"].append(dataset.num_videos)
            self.trainer = Trainer(max_epochs=1,
                                   precision='16-mixed',
                                   accumulate_grad_batches=2,
                                   enable_progress_bar=True,
                                   enable_model_summary=False,
                                   num_sanity_val_steps=0,
                                   callbacks=[lr_monitor, checkpoint_callback])
            self.trainer.fit(model, loader)
            self.competence = self.evaluate_competence(max_epochs, epoch, c0, p)

    def validate(self, model):
#         dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
#                                 target_name=model.target_label,
#                                 transforms=video_transforms)
#         loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
#         self.trainer.validate(model, loader)
        self.trainer.validate(model)

    def test(self, model):
#         dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
#                                 target_name=model.target_label,
#                                 transforms=video_transforms)
#         loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
#         self.trainer.test(model, loader)
        self.trainer.test(model)

### Model

In [ ]:
target_label = 'high_erythrocytes'
num_classes = 2

In [ ]:
class TestModel(LightningModule):
    def __init__(self):
        super(TestModel, self).__init__()
        # model architecture
        self.video_model = torch.hub.load("facebookresearch/pytorchvideo", "efficient_x3d_xs", pretrained=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, num_classes)

        self.lr = 1e-3
        self.batch_size = 8
        self.numworkers = 0
        # evaluation metric
        self.metric = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        # loss function
        self.criterion = nn.CrossEntropyLoss()
        # helpers
        self.target_label = target_label
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.testing_step_outputs = []

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}
    
    # This should remain commented out because in CurriculumTrainer dataloader created in every epoch.
    # Do not uncomment
    # def train_dataloader(self):
    #     dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
    #                           target_name=self.target_label,
    #                           transforms=video_transforms)
    #     loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
    #     return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.training_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_train_epoch_end(self):
        outputs = self.training_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.training_step_outputs = []
        self.log('train/loss', loss)
        self.log('train/metric', metric)
        run["train/loss"].append(loss)
        run["train/metric"].append(metric)

    def val_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=val_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.validation_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.validation_step_outputs = []
        self.log('val/loss', loss)
        self.log('val/metric', metric)
        run["val/loss"].append(loss)
        run["val/metric"].append(metric)
        print({'loss': loss, 'metric': metric})

    def test_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        self.testing_step_outputs.append({'label': label, 'pred': out})
        return {'label': label, 'pred': out}

    def on_test_epoch_end(self):
        outputs = self.testing_step_outputs
        label = torch.cat([x['label'] for x in outputs]).cpu().detach().numpy()
        pred = torch.cat([x['pred'].argmax(dim=1) for x in outputs]).cpu().detach().numpy()
        self.testing_step_outputs = []
        print(classification_report(label, pred))

In [ ]:
model = TestModel()

In [ ]:
params = {
    "n_epochs": 150,
    "c0": 0.05,
    "p": 2,
    # In difficulty function: alpha * df['blur'] + beta * df['noise'] + gamma * df['distances']
    "alpha": 0.5, 
    "beta": 0, # Do not use, used for testing theory
    "gamma": 0.5 
}
run["model/parameters"] = params
run["model/architecture"] = "efficient_x3d_xs"
run["model/difficulty_func"] = "alpha * df['blur'] + beta * df['noise'] + gamma * df['distances']"

In [ ]:
os.chdir("/kaggle/working/")

In [ ]:
# Run this for trainer without curriculum
seed_everything(seed)
trainer = Trainer(max_epochs=params["n_epochs"],
                  precision='16-mixed',
                  accumulate_grad_batches=2,
                  enable_progress_bar=True,
                  num_sanity_val_steps=0,
                  callbacks=[lr_monitor, checkpoint_callback])

dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                      target_name=model.target_label,
                      transforms=video_transforms)
loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
trainer.fit(model, loader)

In [ ]:
# Used for testing theory, ignore that
seed_everything(seed)
for epoch in range(1, params["n_epochs"]+1):
    dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                          target_name=model.target_label,
                          transforms=video_transforms)
    loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
    trainer = Trainer(max_epochs=1,
                           precision='16-mixed',
                           accumulate_grad_batches=2,
                           enable_progress_bar=True,
                           enable_model_summary=False,
                           num_sanity_val_steps=0,
                           callbacks=[lr_monitor, checkpoint_callback])
    trainer.fit(model, loader)

In [ ]:
# Run this for curriculum learning
trainer = CurriculumTrainer()
trainer.fit(model, train_data, 'erythrocytes', 'high_erythrocytes',
            params["n_epochs"], params["c0"], params["p"],
            params["alpha"], params["beta"], params["gamma"])

In [ ]:
model = TestModel.load_from_checkpoint(checkpoint_callback.best_model_path)

In [ ]:
trainer.validate(model)

In [ ]:
trainer.test(model)

In [ ]:
run.stop()